In [1]:
from bs4 import BeautifulSoup
from random import randint, choice
import requests
import pandas as pd

In [2]:
url_billboard = 'https://www.billboard.com/charts/hot-100/'
response = requests.get(url_billboard)
response.status_code

200

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')
result = soup.find_all('div', class_='o-chart-results-list-row-container')


In [4]:
display(result[0].find_all('span')[5].text.strip())


'1'

In [5]:
result[0].h3.find_next('span').get_text().strip()


'Jack Harlow'

In [6]:
songs = []
artists=[]
lastweeks = []
peaks=[]
durations=[]
for res in result:
    song = res.h3.get_text().strip()
    artist = res.h3.find_next('span').get_text().strip()
    lastweek = res.find_all('span')[5].text.strip()
    peak = res.find_all('span')[6].text.strip()
    duration = res.find_all('span')[7].text.strip()
    
    songs.append(song)
    artists.append(artist)
    lastweeks.append(lastweek)
    peaks.append(peak)
    durations.append(duration)

In [7]:
hot100 = pd.DataFrame({'song': songs, 'artist': artists, 
                       'Last Week Ranking': lastweeks, 'Peak Ranking': peaks,
                        'Weeks on Chart': durations})

In [8]:
hot100.to_csv('BillboardHot100.csv')
hot100

,song,artist,Last Week Ranking,Peak Ranking,Weeks on Chart
0,Lovin On Me,Jack Harlow,1,1,14
1,Texas Hold 'Em,Beyonce,2,1,-
2,Carnival,¥$: Kanye West & Ty Dolla $ign,3,1,-
3,Beautiful Things,Benson Boone,3,3,4
4,Lose Control,Teddy Swims,2,2,27
...,...,...,...,...,...
95,Mmhmm,BigXthaPlug,83,65,8
96,Monaco,Bad Bunny,77,5,18
97,Prove It,21 Savage & Summer Walker,75,43,5
98,Perro Negro,Bad Bunny & Feid,82,20,15


In [9]:
url_popvortex = 'http://www.popvortex.com/music/charts/top-100-songs.php'
response = requests.get(url_popvortex)
response.status_code

200

In [10]:
soup = BeautifulSoup(response.text, 'html.parser')
result = soup.find_all('div', class_="chart-content col-xs-12 col-sm-8")
#for res in result:
#    print(res.prettify())

In [11]:
songs=[]
artists=[]
genres=[]
dates= []
for res in result:
    song = res.cite.text.strip()
    artist = res.em.text.strip()
    genre = res.a.text.strip()
    date = res.li.find_next_sibling().text.split(":")[1]
    songs.append(song)
    artists.append(artist)
    genres.append(genre)
    dates.append(date)

In [12]:
popvortex = pd.DataFrame({'song': songs, 'artist': artists, 'genre': genres, 'release_date':dates})

In [13]:
popvortex.to_csv('Popvortex100.csv')

In [14]:
popvortex

,song,artist,genre,release_date
0,TEXAS HOLD 'EM,Beyoncé,Country,"February 11, 2024"
1,Lose Control,Teddy Swims,Pop,"June 23, 2023"
2,Beautiful Things,Benson Boone,Pop,"January 19, 2024"
3,TEXAS HOLD 'EM,Beyoncé,Country,"February 9, 2024"
4,dont let me go,mgk,New Release,Hip-Hop / Rap
...,...,...,...,...
95,Is It Over Now? (Taylor's Version) [From The V...,Taylor Swift,Pop,"October 27, 2023"
96,Scared To Start,Michael Marcagi,Alternative,"January 12, 2024"
97,Karma,Taylor Swift,Pop,"October 21, 2022"
98,CARNIVAL,"¥$, Kanye West & Ty Dolla $ign",Hip-Hop / Rap,"February 8, 2024"


In [26]:
def getsong(df):
    song = input("Song:\n").lower()
    if song in df['song'].str.lower().values:
        print('TRUE')
        return True
    else:
        print('FALSE')
        return False

In [29]:
a = getsong(hot100)

Song:
carnaval
FALSE


In [32]:
def recommendation(df1, df2):
    def getsong1(song, df):
        return song in df['song'].str.lower().values
    
    user_input = input("Song:\n").lower()
    if getsong1(user_input, df1):
        df = df1[df1['song'] != user_input]
    elif getsong1(user_input, df2):
        df = df2[df2['song'] != user_input]
    else:
        print("Error: The provided song is not found in our database.")
        return
    
    i = randint(0, len(df) - 1)
    rec = df.iloc[i]
    song = rec['song']
    artist = rec['artist']
    print(f"Recommended Song: {song} by {artist}")

In [33]:
recommendation(hot100,popvortex)

Song:
carnival
Recommended Song: Burn It Down by Parker McCollum


In [34]:
def recommendation2(df1, df2):
    user_input = input("Song:\n").lower()
    
    df1['song'] = df1['song'].str.lower()
    df2['song'] = df2['song'].str.lower()
    
    if user_input in df1['song'].values:
        df = df1[df1['song'] != user_input]
    elif user_input in df2['song'].values:
        df = df2[df2['song'] != user_input]
    else:
        print("Error: The provided song is not found in our database.")
        return
    
    if user_input in df1['song'].values and user_input in df2['song'].values:
        df = pd.concat([df1, df2], ignore_index=True)
    
    rec = choice(df.index)
    song = df.loc[rec, 'song']
    artist = df.loc[rec, 'artist']
    print(f"Recommended Song: {song} by {artist}")

In [35]:
recommendation2(hot100,popvortex)

Song:
carnival
Recommended Song: country's cool again by Lainey Wilson
